#### Import the libraries

In [ ]:
!pip install fuzzymatcher

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 20.4 MB/s eta 0:00:00
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13901 sha256=80d02ff9b0b1ff205eba71269e9f2bcba61231b5773978154bce36725af7b245
  Stored in directory: /root/.cache/pip/wheels/23/dd/1d/6cdd346605db62bde1f60954155e9ce48f4681c243f265b704
Successfully built metaphone


In [ ]:
import pandas as pd
import fuzzymatcher

#### Import the files required

In [ ]:
# Extracting training folder from google drive into colab
!gdown https://drive.google.com/drive/folders/1qoQQEqB_nOGhFklDfIv5SlUqOB3m0H-m --quiet --folder

# Extracting testing folder from google drive into colab
!gdown https://drive.google.com/drive/folders/1qt0tAS5qNDHJxEwXuNxKQa_EP7YT9By7 --quiet --folder

# Extracting validation folder from google drive into colab
!gdown https://drive.google.com/drive/folders/1R_zQhkV8QyCHiMAKgremlsM3aS-JmlsV --quiet --folder

In [ ]:
import gdown

# Replace 'FILE_ID' with the actual file ID from your Google Drive
file_id = '1qiHvk1noLLppqRL2fckAQ5MyEjd_HXQ1'

# Replace 'YOUR_FILE_NAME.xlsx' with the desired name for the downloaded file
output_file_name = 'credit_bureau.xlsx'

# Construct the download link
download_link = f'https://drive.google.com/uc?id={file_id}'

# Download the file using gdown
gdown.download(download_link, output_file_name)

print(f'{output_file_name} downloaded successfully!')

Downloading...
From: https://drive.google.com/uc?id=1qiHvk1noLLppqRL2fckAQ5MyEjd_HXQ1
To: /content/credit_bureau.xlsx
100%|██████████| 9.44M/9.44M [00:00<00:00, 51.2MB/s]


credit_bureau.xlsx downloaded successfully!


#### Define a function to convert the data to pandas DataFrame

In [ ]:
def construct_path_and_df(data_type):
    base_path = "/content"
    data_file_name = f"{data_type} Data.xlsx"
    gt_file_name = f"{data_type} Data - Ground Truth.xlsx"
    data_file_path = f"{base_path}/{data_type}/{data_file_name}"
    gt_file_path = f"{base_path}/{data_type}/{gt_file_name}"

    data_df = pd.read_excel(data_file_path)
    try:
        gt_df = pd.read_excel(gt_file_path)
    except FileNotFoundError:
        gt_df = None

    return data_df, gt_df

In [ ]:
train_df,train_labels = construct_path_and_df("Training")
test_df,test_labels = construct_path_and_df("Test")
valid_df,_ = construct_path_and_df("Validation")

In [ ]:
credit_df = pd.read_excel("/content/credit_bureau.xlsx")

#### Fuzzymatch maker for record matching

https://github.com/RobinL/fuzzymatcher

In [ ]:
import pandas as pd
import fuzzymatcher

def populate_fields(row, credit_df):
    if pd.notna(row['id']):
        credit_bureau_id = row['id']
        credit_row = credit_df[credit_df['id'] == credit_bureau_id]
        if not credit_row.empty:
            row['First Name'] = credit_row['first_name'].values[0]
            row['Middle Name'] = credit_row['Middle name'].values[0]
            row['Last Name'] = credit_row['last_name'].values[0]
            row['Prefix'] = credit_row['prefix'].values[0]
            row['Suffix'] = credit_row['suffix'].values[0]
            row['Address Line 1'] = credit_row['addr 1'].values[0]
            row['Address Line 2'] = credit_row['addr 2'].values[0]
            row['City'] = credit_row['city'].values[0]
            row['State'] = credit_row['state'].values[0]
            row['Zip'] = credit_row['zip'].values[0]
            row['DOB'] = credit_row['DOB'].values[0]
    return row


In [ ]:
def perform_fuzzy_matching(data_df, credit_df, similarity_threshold=0.4, output_file=None):
    matched_results = fuzzymatcher.fuzzy_left_join(
        data_df,
        credit_df,
        left_on=data_df.columns.tolist(),
        right_on=credit_df.columns.tolist(),
        left_id_col='Cust ID',
        right_id_col='id'
    )

    filtered_results = matched_results[matched_results['best_match_score'] > similarity_threshold]

    # Populate the new DataFrame with data from either credit_df or train_df
    merged_df = filtered_results.apply(lambda row: populate_fields(row, credit_df), axis=1)

    # Select only the desired columns
    output_columns = ['Cust ID', 'First Name', 'Middle Name', 'Last Name', 'Prefix', 'Suffix',
                      'Address Line 1', 'Address Line 2', 'City', 'State', 'Zip', 'DOB']
    merged_df = merged_df[output_columns].copy()
    # Convert 'DOB' column to datetime format
    merged_df['DOB'] = pd.to_datetime(merged_df['DOB'], errors='coerce')

    # Filter out rows where date conversion failed (invalid or non-date values)
    valid_dob_mask = merged_df['DOB'].notnull()
    merged_df = merged_df[valid_dob_mask]

    # Convert 'DOB' column to yyyy/mm/dd format
    merged_df['DOB'] = merged_df['DOB'].dt.strftime('%Y/%m/%d')

    if output_file:
        merged_df.to_csv(output_file, index=False)

    return merged_df,filtered_results

In [ ]:
similarity_threshold = 0.6

train_merged_df,filtered_results_train  = perform_fuzzy_matching(train_df, credit_df, similarity_threshold, "results_train.csv")
test_merged_df,filtered_results_test = perform_fuzzy_matching(test_df, credit_df, similarity_threshold, "results_test.csv")
valid_merged_df,filtered_results_valid  = perform_fuzzy_matching(valid_df, credit_df, similarity_threshold, "results_valid.csv")

In [ ]:
selected_columns = ['Cust ID', 'id']

train_selected_merged_df = filtered_results_train[selected_columns]

test_selected_merged_df = filtered_results_test[selected_columns]

In [ ]:
valid_selected_merged_df = filtered_results_valid[selected_columns]

In [ ]:
valid_selected_merged_df.to_csv("validation_file", index=False)

#### Accuracy calculation

In [ ]:
# Create a DataFrame from selected_columns
selected_ids = train_selected_merged_df[selected_columns]

# Merge selected_ids with train_labels based on 'Cust ID'
merged_with_labels = pd.merge(selected_ids, train_labels, on='Cust ID', how='left')

# Calculate accuracy
accuracy_train = (merged_with_labels['Credit Bureau ID'] == merged_with_labels['id']).mean()

print(f"Accuracy on train dataset: {accuracy_train:.2%}")


Accuracy on train dataset: 84.62%


In [ ]:
# Create a DataFrame from selected_columns
selected_ids = test_selected_merged_df[selected_columns]

# Merge selected_ids with train_labels based on 'Cust ID'
merged_with_labels = pd.merge(selected_ids, test_labels, on='Cust ID', how='left')

# Calculate accuracy
accuracy_train = (merged_with_labels['Credit Bureau ID'] == merged_with_labels['id']).mean()

print(f"Accuracy on train dataset: {accuracy_train:.2%}")

Accuracy on train dataset: 100.00%
